# Fallstudie - Der Computer lernt Zahlen zu lesen
Wir wollen heute schauen, was wir mit unseren Python-Kenntnissen alles schon tun können. Als Beispiel wollen wir dem Computer Lesen beibringen und überprüfen wie gut er das dann auch macht. Etwas genauer soll der Computer von Hand geschriebene Ziffern erkennen können. Eine bekannte Sammlung solcher Ziffern ist die MNIST Datenbank. Einige Beispiele der Zahlen sind hier:

![Images von der MNIST Datenbank](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

> Bild von [Wikipedia](https://de.wikipedia.org/wiki/MNIST-Datenbank)

Wir werden dazu eine Technik aus dem Maschinellen Lernen einsetzen, die unter dem Namen Deep Learning bekannt ist. Deep Learning revolutioniert seit einigen Jahren die Informatik. Entsprechend gibt es viele Bibliothken, die wir nutzen können. Wir können also auf dem aufbauen, was andere schon programmiert haben.

Wir verwenden heute überwiegend Keras von Tensorflow, ein Deep-Learning Framework, also eine Bibliothek welche die gesamte Funktionalität mibringt, die wir benötigen um Deep Learning umzusetzen. Zudem verwenden wir noch die Bibliotheken welche Sie sich im Selbststudium angeschaut haben.

Auch wenn wir für die Aufgabe heute nicht genau verstehen müssen wie Deep-Learning funktioniert, hier ein kurzer Überblick.



# Deep-Learning kurz erklärt

Die wichtigste Komponente beim Deep learning ist ein sogenanntes neuronales Netz (NN). Wir wollen ein neuronales Netz verwenden um für ein Bild einer Ziffer, die darin gezeigte Ziffer vorher zu sagen. Das neuronale Netz stellt dabei die Bilder numerisch dar und transformiert die Daten, bis die Entscheidung zu welcher Klasse eine Eingabe gehört ganz einfach wird. Grafisch können wir das wie folgt skizzieren:

<center>
<img src="https://drive.google.com/uc?id=1NDIZo9ovJOIKd-wCLzfQNk6NJzUsFpN6" width="500px"/>
</center>

Unser Model wird jedoch etwas anders funktionieren. Das Bild wird als ein langer Vektor, also ein 1D-Array von Zahlen, in das Netz gefüttert. Zudem ist die Vorhersage in unserem Fall nicht nur eine Zahl, sondern ein 1D-Array von zehn Zahlen. Dabei gibt jede Zahl an, welche Ziffer wie wahrscheinlich in dem Bild vorkommt. Die erste Zahl gibt an, wie wahrscheinlich die Ziffer im Bild eine Null ist, die zweite Zahl gibt an, wie wahrscheinlich die Ziffer eine 1 ist, und so weiter. Unser Bild ist also wie folgt schon etwas genauer:

<center>
<img src="https://drive.google.com/uc?id=1Q3lMOMVbgTXOVasNd4O5w0gSIsk7XSoX" width="500px"/>
</center>

Im Innern eines solchen Models gibt es meist einen oder mehrere Layers. Im Namen *Deep-Learning* steckt, dass das Model mehrere Layer "tief" ist.

<center>
<img src="https://drive.google.com/uc?id=1_OXD0SsTDI0sr3Noro8WPkWAGkGJubG9" width="500px"/>
</center>

Für diese Fallstudie wollen wir nicht wissen was genau in den Layern passiert, wer will kann einfach an eine grosse Matrixmultiplikation denken. Diese Layer haben Parameter, oft auch Weights genannt, welche wir während dem Lernen, auch Training genannt, anpassen.

<center>
<img src="https://drive.google.com/uc?id=1Z5bUK4lIS7dVOpWXfAs-uE4e1XGqZPXx" width="500px"/>
</center>

Die Veränderung geschieht mit Hilfe eines Optimierers. Der Optimierer berechnet ein Update für die Gewichte basierend auf dem Fehler, welcher unser Model bei der Vorhersage macht. Der Fehler wird basierend auf der Vorhersage und den wahren Labels durch eine Fehler- oder Loss-Funktion gemessen.

<center>
<img src="https://drive.google.com/uc?id=1v0LjwSUigxN1_ureTIsdtXo-Gh-KJITe" width="500px"/>
</center>

Wir wollen von sehr vielen Bildern lernen. Um dies effizient zu machen, können wir, vorallem auf Grafikkarten, mehrere Bilder auf einmal durch unser neuronales Netz schicken und erhalten für jedes der Bilder eine eigene Antwort. Wir nennen einen solchen Stapel von Bildern welcher auf einmal durch das Netz geschickt wird auch *Batch*.

<center>
<img src="https://drive.google.com/uc?id=1iO7NVINIdPexi8X-gk4GsmLQ1u2iTkUV" width="500px"/>
</center>

So nun haben wir viel über Deep-Learning gelernt und wollen uns diese Sachen in Python anschauen.

## Vorbereitungen und Daten

Wir verwenden hier exemplarisch das Framework Keras, was Teil der Tensorflow Bibliothek ist. Als erstes importieren wir dies. Dass wir nicht jedesmal wenn wir das Jupyter-Notebook erneut ausführen, zufällig ein leicht anderes Resultat bekommen, starten wir immer mit der selben Zufallszahl.

In [ ]:
import tensorflow 
tensorflow.random.set_seed(42)

Nun wollen wir uns die Daten zurecht legen. Wir können die MNIST Daten mit einem einfachen Befehl in das Notebook laden. Wir unterscheiden hier Trainings- und Testdaten. Erstere sind dazu da das Model zu trainieren und damit besser zu machen. Die Testdaten sind danach dazu da, zu schauen wie gut das Model durch das Training geworden ist.

In [ ]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Die Daten sind sogenannte Tensoren, diese können Sie sich wie die Arrays von Numpy vorstellen. Tensoren bieten auch viele zu Numpy Array identische Methoden an.

Wir wollen nun anschauen, wieviele Daten wir geladen haben. Dafür können wir das Attribut `shape` verwenden. Dieses haben Sie schon im Selbstudium kennen gelernt. Das Attribut gibt an, wieviele Elemente das Array entlang jeder Dimension hat.

In [ ]:
print("the shape of the training images:", train_images.shape)
print("the space of the training labels:", train_labels.shape)

Wie wir sehen, haben wir 60000 Bilder für das Training zur Verfügung. Jedes Bild hat die grösse $28\times 28$ pixel. Die Bilder enthalten keine Farben, nur Graustufen.

Wir wollen als nächstes einmal das erste Bild anschauen und das erste Label, also die Information welche Zahl im Bild zu sehen ist, ausgeben. Dafür schreiben wir uns eine Funktion welche das i-te Element der Trainingsdaten mit Hilfe von Pyplot anzeigt und im Titel das Label hinzu schreibt.

Dann rufen wir die Methode mit dem Parameter `0` für das erste Bild auf.

In [ ]:
import matplotlib.pyplot as plt

def show_image(i):
  image = train_images[i,:,:] # TODO: assign the i-th image from train_images
  label = train_labels[i] # TODO: assign the i-th label from train_labels
  # we use the color-map (cm) binary in order to see the numbers as
  # black-on-white and not white-on-black.
  plt.imshow(image, cmap=plt.cm.binary)
  plt.title('the image has the label ' + str(label))

show_image(0) # TODO: call the function show_image to show the first image

Als letzten Schritt mit den Daten wollen wir diese noch so aufbereiten, dass unser Model diese besser verwenden kann. Wir haben besprochen, dass das neuronale Netz jedes Bild als langen Vektor, also als 1D-Array, und nicht als Matrix, bzw. 2d-Array entgegen nimmt. Wir können dafür die Methode `reshape` verwenden, welche als Argument ein Tupel mit den neuen Dimensionen entgegen nimmt. Dabei müssen wir beachten, dass wir die gesamte Anzahl Elemente nicht verändern.

In [ ]:
train_images_input = train_images.reshape((60000, 28*28)) # TODO: reshape the train images
test_images_input = test_images.reshape((10000, 28*28)) # TODO: reshape the test images

# scaling of the values of the pixels into  0..1, usually helps when training
train_images_input = train_images_input.astype("float32") / 255
test_images_input = test_images_input.astype("float32") / 255

train_images_input.shape # TODO: check the shape of the training images

## Das Neuronale Netz

Als nächstes wollen wir ein neuronales Netz bauen. Dazu erstellen wir die einzelnen Layer und fügen diese dann in einer Reihe zu einem Model zusammen. Die Details müssen Sie nicht verstehen. Es reicht wenn Sie sehen, dass wir verschiedene Elemente zu etwas grösserem kombinieren.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

layer1 = layers.Dense(512, activation="relu")
# relu is a function with a kink at 0 like the following sketch: _/

outputLayer = layers.Dense(10, activation="softmax")
# softmax normalizes the values between 0.0 and 1.0, such that they sum to 1

model = keras.Sequential([
  layer1,
  outputLayer
])

Als nächstes müssen wir dem Model noch sagen, welchen Optimierungsalgorithmus wir nutzen wollen, welche Fehlerfunktion wir optimieren wollen und wie wir die Güte des Models messen. Darauf gehen wir hier nicht näher ein, sondern verwenden einfach das folgende.

In [ ]:
model.compile(
  optimizer="rmsprop",
  loss="sparse_categorical_crossentropy",
  metrics=["accuracy"]
)


Nun können wir unser Model trainieren. Dazu verwenden wir die Methode `fit`. Diese bekommt die Bilder und Labels fürs Training. Zusätzlich geben wir noch an wieviele Epochen `epochs` wir trainieren, also wie oft wir alle Trainingsdaten durch unser Model schicken. Als letzts geben wir noch an wie gross die Batchgrösse `batch_size` ist, also wieviele Bilder wir auf einmal durch unser Netzwerk schicken. Zurück bekommen wir den Verlauf des Trainings. Den wir danach visualisieren können.

In [ ]:
history = model.fit(train_images_input,train_labels, epochs=5, batch_size=128)

Wir können nun anschauen wie das Netzwerk lernt, also wie es mit der Zeit immer weniger Fehler macht.

In [ ]:
history.history.keys()
evolution_of_loss_values = history.history["loss"]
evolution_of_loss_values
plt.plot(evolution_of_loss_values) # TODO: simply plot the values to see it graphically

Noch bis vor dem Training war dem Model nicht klar wie gross die Eingabedaten sind. Deshalb konnten wir auch noch nicht schauen wie unser Netzwerk genau aussieht. Nun da die Grösse der Daten welche wir dem Netzwerk füttern bekannt ist, können wir schauen wie unser Model ausschaut und wieviele Parameter, also einzelne Zahlen wir gelernt haben.

In [ ]:
model.summary()

Sie sehen, wir haben in diesem einfachen Model schon fast eine halbe Million Parameter. Grosse Netze welche in realen Szenarien angewendet werden, haben dann oft noch viel mehr Parameter. Diese da sie oft viel tiefer sind, also aus mehr Layern bestehen. 

## Testen und Analysieren

Nachdem das Netzwerk trainiert ist, wollen wir nun untersuchen wie gut unser Model neue Ziffern, welche nicht während dem Training gesehen wurden, klassifiziert werden können.

Dafür schauen wir zuerst an, wie wir unser Model verwenden können und was es für ein Bild der Testdaten zurück gibt.

In [ ]:
predictions = model.predict(test_images_input)
print(predictions.shape) # TODO: show the size of the tensor
print('Vorhersage für das erste Bild',predictions[0,:]) # TODO: print the prediction for the first image

Für jedes Bild bekommen wir eine Wahrscheinlichkeit pro Ziffer. Diese gibt an wie wahrscheinlich es die jeweilige Ziffer ist. Wir können mit der Methode `argmax()` schauen, welches die Ziffer ist, die nach der Meinung unseres Netzes am wahrscheinlichsten im Bild vorkommt.

In [ ]:
predicted = predictions.argmax(axis=1)
print('Vorhergesagte Ziffer für das erste Bild', predicted[0])


Für die Visualisierung der Vorhersage können wir uns erneut eine Funktion schreiben. Dies können wir analog zu der Funktion machen, welche wir verwendet haben um die Trainingsdaten anzuzeigen.

In [ ]:
def show_prediction(i):
  image = test_images[i,:,:] # TODO: assign image
  gt_label = test_labels[i] # TODO: assign the test label
  predicted_label = predicted[i] # TODO: assign the predicted label
  plt.imshow(image, cmap=plt.cm.binary)
  plt.title("image {} : label {} prediction {}".format(i,gt_label, predicted_label))
  plt.show()

show_prediction(0)


Nun können wir schauen ob wir öfters das richtige Label vorher sagen. Dafür schauen wir uns einmal die ersten 20 Vorhersagen an.

In [ ]:
# TODO: print the first 20 training labels and predicted labels
for i in range(0,20):
  print("image {} : label {} preduction {}".format(i,test_labels[i], predicted[i]))

Machen wir wirklich keine Fehler?

In [ ]:
show_prediction(247)

Um das Netzwerk mit allen Testdaten zu evaluieren, gibt es schon eine Evaluationsmethode.

In [ ]:
test_loss, test_acc = model.evaluate(test_images_input, test_labels)
print('test_acc: {}'.format(test_acc))

## Fehlerfälle Visualisieren

Wir zeigen nun noch alle Fehlerfälle an. Können Sie die Fehler nachvollziehen?

In [ ]:
import IPython
import time
%matplotlib inline

# TODO: show prediction for error cases in a loop (add two lines to your loop)
for i in range(0,test_labels.size):
  if not test_labels[i] == predicted[i]:
    show_prediction(i)
    IPython.display.clear_output(wait=True) # necessary to update the image
    time.sleep(2) # wait for 2 seconds before you continue

## Miniübungen
- Trainieren Sie das Model für mehr Iterationen.
- Erweitern Sie das Model und fügen Sie mehr Layer ein. Reduzieren sich die Fehler im Testset mit mehr Layern?
- Fügen Sie nach jedem `Dense`-Layer, ausser dem Output-Layer, einen Layer ein welchen Sie mit `layers.Dropout(0.2)` erstellen.
- Wenn Sie wollen können Sie nun noch etwas genauer analysieren ob das Netzwerk gewisse Ziffern öfters verwechselt als andere. Dafür können Sie für jedes Label anschauen wie oft welche andere Ziffer falsch vorhergesagt wird. Wenn Sie dies in eine Numpy Array speichern, können Sie das Ergebnis mit Pyplot plotten. Dafür starten Sie mit einem zweidimensionalen Numpy Array gefüllt mit Nullen. Bei jedem Fehler erhöhen Sie die Zahl im Array an der Stelle (gt_label,prediction).

In [ ]:
layer1 = layers.Dense(512, activation="relu")
layer2 = layers.Dense(512, activation="relu")
dropout = layers.Dropout(0.2) # simulates missing/noisy data

model2 = keras.Sequential([
  layer1,
  dropout,
  layer2,
  dropout,
  outputLayer
])

model2.compile(
  optimizer="rmsprop",
  loss="sparse_categorical_crossentropy",
  metrics=["accuracy"]
)

history = model2.fit(train_images_input,train_labels, epochs=5, batch_size=128)

predictions = model.predict(test_images_input)

predicted = predictions.argmax(axis=1)

show_prediction(247)

test_loss, test_acc = model2.evaluate(test_images_input, test_labels)
print('test_acc: {}'.format(test_acc))

In [ ]:
import numpy as np
error_counter = 0
confusion_errors_matrix = np.zeros(shape = [10,10])

for i in range(0,10000):
  label = test_labels[i]
  prediction = predicted[i]
  if not label == prediction:
    error_counter += 1
    confusion_errors_matrix[label,prediction] += 1
    print('\r','E{} I{} : label {} prediction {}'.format(error_counter, i, label, prediction))
  else:
    print('\r','I{} : label {} prediction {}'.format(i, label, prediction), end='')
print()
print('the network had {} errors out of 10000'.format(error_counter))

Wir schauen an welche Kombinationen von Label-Prediction die meisten Fehler erzeugt. Dies gibt ein Hinweis, was für das Netzwerk schwer zu lernen ist.

In [ ]:
plt.imshow(confusion_errors_matrix)
plt.ylabel('label')
plt.xlabel('prediction')